In [1]:
import os
import pandas as pd
import geopandas as gpd
import plotly.express as px
import plotly.offline as pyo

# Set notebook mode to work in offline
pyo.init_notebook_mode()


In [ ]:
ita_regions = r"..\Data\ISTAT\Reg01012021_g_WGS84.shp"
population_file = r"..\Data\ISTAT\pop2020.csv"

In [ ]:
# Read Italian regions
ita_regions_df = gpd.read_file(ita_regions)
unique_regions = ita_regions_df[["DEN_REG", "geometry"]].groupby("DEN_REG").first()

In [ ]:
# Read population file
pop_df = pd.read_csv(population_file)

# Join with regions
pop_df = pd.merge(ita_regions_df, pop_df, left_on='DEN_REG', right_on='Regione')
pop_df = pop_df[(pop_df["eta"]!="Totale")]
pop_df["eta"] = pop_df["eta"].astype(int)

# Total population
pop_df["POP"] = pop_df["maschi"] + pop_df["femmine"]

# Calculate aging index
pop15 = pop_df[(pop_df["eta"] < 15)]
pop65 = pop_df[(pop_df["eta"] > 65)]
pop15 = pop15[["Regione", "POP"]].groupby("Regione").sum("POP")
pop65 = pop65[["Regione", "POP"]].groupby("Regione").sum("POP")
aging = (pop65/pop15) * 100
aging = aging.reset_index()
aging = pd.merge(aging, unique_regions, left_on='Regione', right_on='DEN_REG')
aging["year"] = int(os.path.basename(population_file)[3:7])
aging = aging.rename({'POP': 'Aging Index'}, axis=1)
aging = gpd.GeoDataFrame(aging, crs=ita_regions_df.crs).to_crs("epsg:4326")

In [ ]:
lon_min, lat_min, lon_max, lat_max = aging.total_bounds
cx, cy = (lon_min+lon_max)/2, (lat_min+lat_max)/2

In [ ]:
title = '<b>2020 Aging Index by Region (Italy)</b><br>'
title += '<i style="font-size:12px">Number of elderly population (aged 65 years and over) per 100 individuals younger than 14 years old</i>'

In [ ]:
# Save the map
fig = px.choropleth_mapbox(aging, 
                    geojson=aging.geometry, 
                    locations=aging.index,
                    color="Aging Index",
                    zoom=4,
                    center={'lat': cy, 'lon': cx},
                    hover_data =["Regione", "Aging Index"],
                    mapbox_style="carto-positron",
                    color_continuous_scale="Greys")
fig.update_layout(
    title_text = title,
    annotations = [dict(
        x=0.0,
        y=0.0,
        xref='paper',
        yref='paper',
        text='Source: <a href="https://demo.istat.it/popres/download.php?anno=2020&lingua=ita">ISTAT</a>',
        showarrow = False
    )]
)

fig.update_geos(fitbounds="locations", visible=False)
fig.write_html("14_NewTool.html")